# Importações

In [ ]:
import os
import kagglehub
import pandas as pd
from google.cloud import storage

# Ambiente de Execução: Colab X VSCode

In [ ]:
# # Usar no Google Colab

# from google.colab import auth
# auth.authenticate_user()
# path_bronze = '/content/bronze/'

# path_bronze

In [ ]:
# Usar no VSCode

path_bronze = os.path.abspath('../data/temp/bronze')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../gcp_key.json"

path_bronze

# Baixar os arquivos do Kaggle e mover para a pasta do projeto

In [ ]:
kaggle_owner = 'stoney71'
kaggle_dataset = 'new-york-city-transport-statistics'

def download_kaggle_dataset(path_bronze, kaggle_owner, kaggle_dataset):
    print('Iniciando download dos arquivos do Kaggle')
    path_origin = kagglehub.dataset_download(os.path.join(kaggle_owner, kaggle_dataset)) # Baixa os arquivos de GPS do Kaggle
    
    os.system(f'mkdir -p {path_bronze}') # Cria a para que conterá os arquivos originais baixados

    # Copia todos os arquivos da pasta do Kaggle para data_raw
    os.system(f'mv {path_origin}/* {path_bronze}/')
    os.system(f'rm -rf {path_origin}')
    
    print(f'Arquivos baixados e movidos para {path_bronze}.')

In [ ]:
download_kaggle_dataset(path_bronze, kaggle_owner, kaggle_dataset)

# Salvar os arquivos originais no Cloud na primeira camada de dados: 'data/bronze'

In [ ]:
def extract_year_month(date): # funcao que transformar uma data em ano e mes
    return date.strftime('%y%m')

In [ ]:
year_month = extract_year_month(pd.to_datetime('2017-06-01'))  # Exemplo de uso da função, como se fosse o dia 01 de junho de 2017

print(f'Year-Month: {year_month}')

In [ ]:
def upload_to_bucket_bronze(path_bronze, file_name_bronze): # Faz o upload de todos os arquivos para o bucket
    print('Iniciando upload do arquivo para a Cloud.')
    client = storage.Client() # Cria o cliente para acessar o bucket

    bucket = client.bucket('etl_bus_gps') # Referência pro bucket do GCP
    path_folder_file_name_bronze = os.path.join(path_bronze, file_name_bronze)

    blob = bucket.blob(f'bronze/temp/{file_name_bronze}')
    blob.upload_from_filename(path_folder_file_name_bronze)
    print(f'Arquivo "{file_name_bronze}" enviado para a pasta "/bronze" no bucket "{bucket.name}"')

In [ ]:
file_name_bronze = f'mta_{year_month}.csv'

upload_to_bucket_bronze(path_bronze, file_name_bronze)  # Faz o upload do arquivo para o bucket

# Pipeline de Execução

In [ ]:
def pipeline_bronze(year_month):
    file_name_bronze = f'mta_{year_month}.csv'  # Define o nome do arquivo bronze com base no ano e mês
    path_bronze = os.path.abspath('../data/temp/bronze')
    
    upload_to_bucket_bronze(path_bronze, file_name_bronze)  # Faz o upload do arquivo para o bucket
    
    print(f'Pipeline bronze concluído para o arquivo: {file_name_bronze}\n')

In [ ]:
pipeline_bronze(year_month)  # Executa a pipeline bronze

In [ ]:
year_month_list = [] # Lista para armazenar os anos e meses

year_month_list.append(extract_year_month(pd.to_datetime('2017-06-01'))) # Adiciona o ano e mês de junho de 2017
year_month_list.append(extract_year_month(pd.to_datetime('2017-08-01'))) # Adiciona o ano e mês de agosto de 2017
year_month_list.append(extract_year_month(pd.to_datetime('2017-10-01'))) # Adiciona o ano e mês de outubro de 2017
year_month_list.append(extract_year_month(pd.to_datetime('2017-12-01'))) # Adiciona o ano e mês de dezembro de 2017

year_month_list

In [ ]:
path_bronze = os.path.abspath('../data/temp/bronze')
kaggle_owner = 'stoney71'
kaggle_dataset = 'new-york-city-transport-statistics'
download_kaggle_dataset(path_bronze, kaggle_owner, kaggle_dataset)

for year_month in year_month_list:
    pipeline_bronze(year_month)  # Executa o pipeline para o ano e mês especificados